<h1 align="center" style="background-color:#616161;color:white">Training and Test Set Selection</h1>

There will be two types of assessments: 
* Predicting what and when known users would like to listen to next
* Predicting what and when previously unknown users would like to listen to next

<b>Predicting next action of a known user:</b>
* For every user 3 randomly selected time slots will be chosen from the last 10% of their listening history
    * A timeslot in the 10-30 minute interval range. This is to assess
    * A timeslot in the 12 hours+ interval range
    * A timeslot in the 10minutes + interval range. This is to act as a control sample to assess whether the interval range do make a difference in prediction.
    
<b>Unknown users sample:</b>
* 10 users will be randomly selected and separated out to be used in evaluating unknown users

<h3 style="background-color:#616161;color:white">0. Setup code</h3>

In [6]:
# import modules
%matplotlib inline
import os
import sys
import numpy as np
import pandas as pd
import datetime
import csv
import json
import random
import sqlite3
from pathlib import Path
from sklearn.manifold import TSNE
from sklearn.datasets import fetch_20newsgroups
import matplotlib.pyplot as plt
#import gensim, logging

## Parameters you can change

# Abs path to settings file
#root = "C:/DS/Github/MusicRecommendation"  # BA, Windows
root = "/home/badrul/git/EventPrediction"  # BA, Linux

## Import the codebase module
fPath = root + "/1_codemodule"
if fPath not in sys.path: sys.path.append(fPath)

import coreCode as cb
import lastfmCode as fm

## Finish setting up
os.chdir(root)

settingsDict =  cb.loadSettings()

dbPath = root + settingsDict['mainDbPath']
fmSimilarDbPath = root + settingsDict['fmSimilarDbPath']
fmTagsDbPath = root + settingsDict['fmTagsDbPath']
trackMetaDbPath = root + settingsDict['trackmetadata']


<h3 style="background-color:#616161;color:white">1. Model Parameters</h3>

In [7]:
# Eval 1: Adapting to new users
newUsers = 10   # Num of randomly selected users to separate out of eval 2

# Eval 2: Random periods
randomPeriods = 2 # Num of random periods per user

<h3 style="background-color:#616161;color:white">2. Eval I: Select random periods</h3>

In [5]:
sqlStr = "Select UserID, historyID from tblMain where MinsSincePrevPlay > {} Group by UserID, historyID".format(StartIntervalTime)
con = sqlite3.connect(dbPath)
df = pd.read_sql_query(sqlStr, con)
dfUsers = df['UserID'].unique()


for userID in dfUsers:   # userID is the df index
    # Filter df or each user
    df2 = df[df.UserID == userID]
    s = df2.shape
    cutoff=random.randint(0, s[0])
    sqlStr = "Update tblUsers set Testcutoff = {} where userID = {}".format(cutoff,userID)
    try:
        con.execute(sqlStr)
        
    except:
        con.close()
        print(sqlStr)
        print ("Unexpected error:", sys.exc_info()[0])
        raise
    
    
con.commit()
con.close()

NameError: name 'StartIntervalTime' is not defined

<h3 style="background-color:#616161;color:white">3. Eval II: Select random 'new users'</h3>

In [8]:
con = sqlite3.connect(dbPath)

# First reset back to 0
con.execute("Update tblUsers Set TestUser = 0")
con.commit()

# Select random users
sqlStr= "SELECT UserID FROM tblUsers Group by UserID ORDER BY RANDOM() LIMIT {}".format(newUsers+1)

#newUsersList = pd.read_sql_query(sqlStr, con)
#for row in newUsersList.itertuples():
#    sqlStr = "Update tblUsers Set TestUser = 1 where UserID = {}".format(row[1])
#    con.execute(sqlStr)

newUsersList= [93,59,132,142,887, 543,571, 639, 525, 820, 813]
for user in newUsersList:
    sqlStr = "Update tblUsers Set TestUser = 1 where UserID = {}".format(user)
    con.execute(sqlStr)

    
con.commit()
con.close()

In [ ]:
newUsersList